# Exemplo Básico de Uso - Cluster Fácil

Este notebook demonstra os passos essenciais para usar a biblioteca `cluster-facil`.

## 1. Instalação

Se você ainda não instalou a biblioteca, descomente e execute a célula abaixo. Ela instalará a versão mais recente diretamente do repositório GitHub.

**Observação:** Substitua `seu-usuario` pelo nome de usuário correto do proprietário do repositório no GitHub.

In [ ]:
# !pip install git+https://github.com/seu-usuario/cluster-facil.git

## 2. Importações e Dados de Exemplo

Vamos importar o `pandas` para criar nosso DataFrame de exemplo e a classe `ClusterFacil` da nossa biblioteca.

In [ ]:
import pandas as pd
from cluster_facil import ClusterFacil

# Criando um DataFrame de exemplo simples com textos
data = {'id': [1, 2, 3, 4, 5, 6, 7, 8],
        'texto_documento': [
            "O juiz determinou o pagamento da indenização.",
            "Recurso de apelação foi interposto pela defesa.",
            "A sentença julgou procedente o pedido do autor.",
            "A corte negou provimento ao recurso especial.",
            "Processo arquivado por falta de movimentação.",
            "Acordo homologado entre as partes.",
            "Indenização por danos morais foi concedida.",
            "Extinção do processo sem resolução de mérito."
        ]}
df_exemplo = pd.DataFrame(data)

print("DataFrame de Exemplo:")
display(df_exemplo)

## 3. Inicializando o ClusterFacil

Criamos uma instância da classe, passando nosso DataFrame.

In [ ]:
cf = ClusterFacil(df_exemplo)

## 4. Preparando os Dados e Analisando o Cotovelo

Agora, chamamos o método `preparar`. Ele fará o pré-processamento, calculará o TF-IDF e **exibirá o gráfico do método do cotovelo**.

Precisamos informar:
*   `coluna_textos`: O nome da coluna que contém os textos a serem clusterizados.
*   `limite_k`: O número máximo de clusters (K) que queremos testar no método do cotovelo.

In [ ]:
print("Iniciando a preparação e geração do gráfico do cotovelo...")
cf.preparar(coluna_textos='texto_documento', limite_k=5)
print("Preparação concluída. Analise o gráfico acima para escolher o número de clusters (K).")

## 5. Finalizando a Clusterização

Após analisar o gráfico do cotovelo (o ponto onde a curva "dobra"), escolhemos o número de clusters (K) que parece mais apropriado.

Chamamos o método `finaliza` informando:
*   `num_clusters`: O valor de K escolhido.
*   `prefixo_saida` (opcional): Um prefixo para os nomes dos arquivos CSV e Excel que serão gerados com os resultados.

Para este exemplo, vamos supor que, após analisar o gráfico, decidimos usar **K=3**.

In [ ]:
k_escolhido = 3
prefixo = 'exemplo_notebook'

print(f"Finalizando a clusterização com K={k_escolhido}...")
cf.finaliza(num_clusters=k_escolhido, prefixo_saida=prefixo)

print(f"\nClusterização finalizada! Verifique os arquivos gerados:")
print(f"- {prefixo}_clusters_1.csv")
print(f"- {prefixo}_amostras_por_cluster_1.xlsx")

## 6. Verificando o Resultado

O DataFrame original dentro do objeto `cf` agora possui uma nova coluna (`cluster_1`) com o ID do cluster atribuído a cada documento.

In [ ]:
print("DataFrame com a coluna de cluster:")
display(cf.df)

--- 
Fim do exemplo básico!